# Load data

In [6]:
import os
import sys
import argparse
import pandas as pd
import re


In [32]:
snp_info = pd.read_excel("/Users/petterbystrom/Documents/Applied Bioinformatics/snp_spreadsheet.xlsx", "Supp. table 14", header=1, index_col=0)
taxa_info = pd.read_excel("/Users/petterbystrom/Documents/Applied Bioinformatics/snp_spreadsheet.xlsx", "Supp. table 13", header=1)
headers_path="/Users/petterbystrom/Documents/Applied Bioinformatics/Headers.txt"
annotation_file = "/Users/petterbystrom/Documents/Applied Bioinformatics/Python/GCF_000009065.1/ncbi_dataset/data/GCF_000009065.1/genomic.gtf"

### open Headers.txt and make into list without > and :

In [201]:
fasta = "test.fasta"
def headers_to_list(fasta_file):
    with open(fasta_file, 'r') as file:
        file_contents = file.readlines()
    return sorted([line.replace('>', '').replace(':', '').replace('\n','') for line in file_contents])
print(headers_from_fasta(fasta))

['AAGAANCCGACGACTAAGGTCGNTACGCGACCGACNCCCATNNNNNNATANAGGCTCGNT', 'AAGAANCCGNCGACTNNGGGCNNNACGNCGCCNACGCCCATTTTTGCNTNNNNGCTCGCT', 'AAGATGTCCCCTGCACGCTCCTCCATNGCCATTAGGTACGCAGCNTNGTAGAGNGCNGGG', 'AAGNCGNGCCTCAACGCCTNATNGGNCCNNNNGGGCCNNGTGGCTCANTNAGGNTTACCN', 'AATGCGCTTCCCCTTGNCACAGCCGCCTNNAATATGGCCGCTAGAACCNGGCGNCGGCTC', 'ACAATNGNGACACAAANTNTGGCGCTGCGTNACTAGCGNACGGGGNGAGGCCGNGATCAT', 'ACAGGGCGCNANNGAAANACGNCCTCNCCGGCCCATACNCTGAGTGATTGGGCCGGACCC', 'ACANGGCGNNNNNNNAATACNTNCTCACCGGCCNATACACTGAGNGATTNGNCCNGACCN', 'ACANTCGGGACACAAATTCTGGCNNNGCGTGNNTAGCGGACGGGGTGAGGCCGTGATCAT', 'ACCTNCGCCCCGCGATNTAAAACTANCTNAGAGGNTNNAACCCCCNCGTATTANNNTTTC', 'ACCTNCGCCNCGCGANTTAANACTACCTNAGAGNGTANAACCCCCNCGCACGANNNNTTC', 'ACGCCGTCCCGTNCCANGNNCTGGTAGCGGGCTGCNGCACTCACCNCTGGCCCCTATACC', 'ACGCCGTCNCNTCCCATGCNTTGGNANNGGGNNGCNGNACTCACCCCTNGCCCNNATACC', 'ACGGNCTGGTTGGCTCAGAACNTCCGCTGNCCNCCGGCNNNCCNNCNNACAGACNCCNGT', 'ACNNNGAGCGGCGNGAGCGGCGCGCGAGCGGGAGGAGGTGCNCTGGANCCCGTCCNGNGG', 'ACTNNNANCGGCGNGAGNNGNNNGNGNGCNGGAGGAGG

In [204]:
def headers_from_fasta(fasta_file):
    headers = []
    
    with open(fasta_file, 'r') as fasta_file:
        current_header = None
        for line in fasta_file:
            line = line.strip()
            if line.startswith('>'):
                # If a line starts with '>', it's a header line
                current_header = line
                headers.append(current_header)
    
    return [line.replace('>', '') for line in headers]

In [205]:
print(headers_from_fasta(fasta))

['Reference_gi|16120353|ref|NC_003143.1| Yersinia pestis CO92 chromosome, complete genome', 'Azov38']


### Make set of common elements

In [94]:
def create_regex(prefix):
    return re.compile(fr'^{prefix}(?:\.[A-Za-z0-9]+)?$')

pandas.core.frame.DataFrame

In [24]:
def transform_elements(regex, elements):
    pattern = re.compile(regex)
    transformed_elements = []
    for element in elements:
        match = pattern.match(element)
        if match:
            transformed_elements.append(re.sub(r'\.[A-Za-z0-9]+$', '', element))
        else:
            transformed_elements.append(element)
    return transformed_elements
regex_pattern = r'^[A-Za-z0-9]+(?:\.[A-Za-z0-9]+)?$' # Your regex pattern
'''
# Test data
my_list = ['BED024', 'BED024.A0102', 'BED024.B0234', 'ELW098', 'ELW098.B0421', 'BED024.', 'BED024A0102']


transformed_list = transform_elements(regex_pattern, Headers_from_fasta[200:])
print(Headers_from_fasta[200:])
print("Transformed list:", transformed_list)
'''

'\n# Test data\nmy_list = [\'BED024\', \'BED024.A0102\', \'BED024.B0234\', \'ELW098\', \'ELW098.B0421\', \'BED024.\', \'BED024A0102\']\n\n\ntransformed_list = transform_elements(regex_pattern, Headers_from_fasta[200:])\nprint(Headers_from_fasta[200:])\nprint("Transformed list:", transformed_list)\n'

In [26]:
enteries = set(snp_info.columns.to_list()) & set(taxa_info["Genome ID"].to_list()) & set(transform_elements(regex_pattern,Headers_from_fasta))
print(len(enteries))
unique_to_snpinfo = [element for element in snp_info.columns.to_list() if element not in enteries]
unique_to_taxainfo = [element for element in set(taxa_info["Genome ID"].to_list()) if element not in enteries]
unique_to_Headersfromfasta = [element for element in transform_elements(regex_pattern,Headers_from_fasta) if element not in enteries]
print(sorted(unique_to_snpinfo))
print(sorted(unique_to_taxainfo))
print(sorted(unique_to_Headersfromfasta))

241
['BSK001', 'BSK001-003', 'BSK003', 'Barcelona3031', 'CHE001', 'CO92 Ref.', 'COL1', 'ELW098', 'London-6330', 'London-ES', 'SC2-SPN7', 'TRP002', 'Y.pseudotuberculosis IP32953 (outgroup)']
['Barcelona 3031', 'CHE1', 'COL1', 'ELW098', 'London  6330', 'London 8124/8291/11972', 'SC2_SPN7', 'TRP002']
['BSK001-003.A0101.A0102.A0103-malt', 'Barcelona3031', 'CHE.C1-C2-3-C4', 'COLC1-COLC2a_COLC2b', 'ELW098_JK1548', 'London_EastSmithfield_8124_8291_11972', 'London_StMaryGraces_6330', 'Reference_gi|16120353|ref|NC_003143.1| Yersinia pestis CO92 chromosome, complete genome', 'SC2_SPN7', 'TRP002.A0101-02', 'outgroup_Y.pseudo']


### Make translation dictionary

In [ ]:
modern_ancient_dict={}

### Make dictionary of Genome ID and coresponding decent

In [ ]:
lineage_to_modern_ancient = dict(zip(taxa_info['Genome ID'], taxa_info['Modern/Ancient']))

### Find amount of ambigious sites in SNP info

In [ ]:
rows_without_N = snp_info[~snp_info.apply(lambda row: row.astype(str).str.contains('N', case=False)).any(axis=1)]

### Extract information from annotation file

In [ ]:
with open(annotation_file, 'r') as file:
    file_contents = file.readlines()

In [155]:
Annotation_dict={}
for i in file_contents:
    match_geneid = re.search(r'gene_id "([A-Za-z0-9_]+)"',i)
    match_location = re.search(r'\tgene\t(\d+)\t(\d+)\t',i)
    match_direction = re.search(r'(?<=\t\.\t)([\-\+])(?=\t\.)',i)
    if match_geneid and match_location and match_direction:
        Annotation_dict[match_geneid.group(1)] = [int(match_location.group(1)),int(match_location.group(2)),match_direction.group(1),0,0]
    
print(Annotation_dict)
print(len(Annotation_dict))
print((len(file_contents)-6)/4)

{'YPO_RS01005': [271, 711, '-', 0, 0], 'YPO_RS01010': [804, 1265, '-', 0, 0], 'YPO_RS01015': [1435, 2427, '+', 0, 0], 'YPO_RS01020': [2526, 3992, '-', 0, 0], 'YPO_RS01025': [3996, 5549, '-', 0, 0], 'YPO_RS01030': [5823, 7691, '+', 0, 0], 'YPO_RS01035': [7896, 8315, '+', 0, 0], 'YPO_RS01040': [8366, 9292, '+', 0, 0], 'YPO_RS01050': [9345, 9515, '+', 0, 0], 'YPO_RS01055': [9512, 10936, '-', 0, 0], 'YPO_RS01060': [11016, 11705, '-', 0, 0], 'YPO_RS01070': [12265, 13807, '+', 0, 0], 'YPO_RS01075': [13944, 14019, '+', 0, 0], 'YPO_RS01080': [14272, 17178, '+', 0, 0], 'YPO_RS01085': [17288, 17403, '+', 0, 0], 'YPO_RS01090': [17906, 19114, '+', 0, 0], 'YPO_RS01095': [19172, 19702, '-', 0, 0], 'YPO_RS01100': [19702, 20289, '-', 0, 0], 'YPO_RS01105': [20443, 20712, '+', 0, 0], 'YPO_RS01110': [20792, 21778, '+', 0, 0], 'YPO_RS01115': [21806, 22429, '+', 0, 0], 'YPO_RS01120': [22921, 25719, '+', 0, 0], 'YPO_RS01125': [26128, 26586, '+', 0, 0], 'YPO_RS01130': [26838, 27488, '-', 0, 0], 'YPO_RS01135'

In [156]:
from Bio import SeqIO
def get_sequence_lengths(fasta_file,header):
    for record in SeqIO.parse(fasta_file, "fasta"):
        if record.id == header[0:len(record.id)]:
            seq_len = len(record.seq)
    return seq_len

In [130]:
print(get_sequence_lengths("test.fasta","Reference_gi|16120353|ref|NC_003143.1| Yersinia pestis CO92 chromosome, complete genome"))

3533


### Make dictioray of gene location and open reading frame

In [157]:
def get_ORF(Annotation_dict):
    for key in Annotation_dict:
        if Annotation_dict[key][2] == "+":
            orf = Annotation_dict[key][0] % 3 + 1
            Annotation_dict[key][3]=orf
            
        elif Annotation_dict[key][2] == "-":
            orf = -1*(((get_sequence_lengths("test.fasta","Reference_gi|16120353|ref|NC_003143.1| Yersinia pestis CO92 chromosome, complete genome")-Annotation_dict[key][1]))%3+1)
            Annotation_dict[key][3]=orf
        #print(key, Annotation_dict[key])
    return Annotation_dict



In [158]:
print(get_ORF(Annotation_dict))


{'YPO_RS01005': [271, 711, '-', -3, 0], 'YPO_RS01010': [804, 1265, '-', -1, 0], 'YPO_RS01015': [1435, 2427, '+', 2, 0], 'YPO_RS01020': [2526, 3992, '-', -1, 0], 'YPO_RS01025': [3996, 5549, '-', -1, 0], 'YPO_RS01030': [5823, 7691, '+', 1, 0], 'YPO_RS01035': [7896, 8315, '+', 1, 0], 'YPO_RS01040': [8366, 9292, '+', 3, 0], 'YPO_RS01050': [9345, 9515, '+', 1, 0], 'YPO_RS01055': [9512, 10936, '-', -2, 0], 'YPO_RS01060': [11016, 11705, '-', -1, 0], 'YPO_RS01070': [12265, 13807, '+', 2, 0], 'YPO_RS01075': [13944, 14019, '+', 1, 0], 'YPO_RS01080': [14272, 17178, '+', 2, 0], 'YPO_RS01085': [17288, 17403, '+', 3, 0], 'YPO_RS01090': [17906, 19114, '+', 3, 0], 'YPO_RS01095': [19172, 19702, '-', -2, 0], 'YPO_RS01100': [19702, 20289, '-', -3, 0], 'YPO_RS01105': [20443, 20712, '+', 2, 0], 'YPO_RS01110': [20792, 21778, '+', 3, 0], 'YPO_RS01115': [21806, 22429, '+', 3, 0], 'YPO_RS01120': [22921, 25719, '+', 2, 0], 'YPO_RS01125': [26128, 26586, '+', 2, 0], 'YPO_RS01130': [26838, 27488, '-', -1, 0], 'YPO

In [165]:
print(snp_info.head(10))

         CO92 Ref. BSK001 BSK003 BSK001-003 LAI009 TRP002 London-ES  \
Position                                                              
17               C      N      N          N      N      N         N   
855              G      .      .          .      .      .         .   
1166             C      .      .          .      .      .         .   
1315             C      N      N          N      N      N         N   
1333             G      .      N          .      .      N         N   
1656             G      .      N          .      .      .         .   
2052             A      .      .          .      .      .         .   
2873             G      .      .          .      .      N         .   
3270             C      .      N          .      .      .         .   
4089             C      N      N          N      N      N         .   

         Barcelona3031 OSL1 NAB003  ... 4.ANT_1454 4.ANT_338 4.ANT_517  \
Position                            ...                                  

Slower solution

In [187]:
def codon_finder(alignment_file, snp_df, Annotation_dict):
    temp_dict = Annotation_dict.copy()
    for key in Annotation_dict:
        temp_dict[key][4] = []
        snp_search =(snp_df.index >= Annotation_dict[key][0]) & (snp_df.index <= Annotation_dict[key][1])
        for i,j in enumerate(snp_search):

            if j==True:
                temp_dict[key][4].append (snp_df.index[i])
            
    return temp_dict

Faster solution

In [193]:
def codon_finder2(alignment_file, snp_df, Annotation_dict):
    for key, value in Annotation_dict.items():
        snp_indices = snp_df[snp_df.index.to_series().between(value[0], value[1])].index.tolist()
        value[4] = snp_indices
    return Annotation_dict

In [189]:
temp_dict = codon_finder("test.fasta",snp_info["Azov38"].to_frame(),Annotation_dict)

print(temp_dict)



{'YPO_RS01005': [271, 711, '-', -3, []], 'YPO_RS01010': [804, 1265, '-', -1, [855, 1166]], 'YPO_RS01015': [1435, 2427, '+', 2, [1656, 2052]], 'YPO_RS01020': [2526, 3992, '-', -1, [2873, 3270]], 'YPO_RS01025': [3996, 5549, '-', -1, [4089, 4225]], 'YPO_RS01030': [5823, 7691, '+', 1, [6103]], 'YPO_RS01035': [7896, 8315, '+', 1, []], 'YPO_RS01040': [8366, 9292, '+', 3, [8793]], 'YPO_RS01050': [9345, 9515, '+', 1, []], 'YPO_RS01055': [9512, 10936, '-', -2, []], 'YPO_RS01060': [11016, 11705, '-', -1, [11330]], 'YPO_RS01070': [12265, 13807, '+', 2, []], 'YPO_RS01075': [13944, 14019, '+', 1, []], 'YPO_RS01080': [14272, 17178, '+', 2, []], 'YPO_RS01085': [17288, 17403, '+', 3, []], 'YPO_RS01090': [17906, 19114, '+', 3, []], 'YPO_RS01095': [19172, 19702, '-', -2, []], 'YPO_RS01100': [19702, 20289, '-', -3, [19828, 20252]], 'YPO_RS01105': [20443, 20712, '+', 2, []], 'YPO_RS01110': [20792, 21778, '+', 3, [21219]], 'YPO_RS01115': [21806, 22429, '+', 3, []], 'YPO_RS01120': [22921, 25719, '+', 2, [22

In [194]:
temp_dict2 = codon_finder2("test.fasta",snp_info["Azov38"].to_frame(),Annotation_dict)

print(temp_dict2)

{'YPO_RS01005': [271, 711, '-', -3, []], 'YPO_RS01010': [804, 1265, '-', -1, [855, 1166]], 'YPO_RS01015': [1435, 2427, '+', 2, [1656, 2052]], 'YPO_RS01020': [2526, 3992, '-', -1, [2873, 3270]], 'YPO_RS01025': [3996, 5549, '-', -1, [4089, 4225]], 'YPO_RS01030': [5823, 7691, '+', 1, [6103]], 'YPO_RS01035': [7896, 8315, '+', 1, []], 'YPO_RS01040': [8366, 9292, '+', 3, [8793]], 'YPO_RS01050': [9345, 9515, '+', 1, []], 'YPO_RS01055': [9512, 10936, '-', -2, []], 'YPO_RS01060': [11016, 11705, '-', -1, [11330]], 'YPO_RS01070': [12265, 13807, '+', 2, []], 'YPO_RS01075': [13944, 14019, '+', 1, []], 'YPO_RS01080': [14272, 17178, '+', 2, []], 'YPO_RS01085': [17288, 17403, '+', 3, []], 'YPO_RS01090': [17906, 19114, '+', 3, []], 'YPO_RS01095': [19172, 19702, '-', -2, []], 'YPO_RS01100': [19702, 20289, '-', -3, [19828, 20252]], 'YPO_RS01105': [20443, 20712, '+', 2, []], 'YPO_RS01110': [20792, 21778, '+', 3, [21219]], 'YPO_RS01115': [21806, 22429, '+', 3, []], 'YPO_RS01120': [22921, 25719, '+', 2, [22

Find bases in codon

In [195]:
def find_bases(Annotation_dict, fasta_file):
    codons=[]
    for key,value in Annotation_dict.items():
        position = 0
        if len(value[-1])>0:
            # + reading frame
            if value[2] == "+":
                for i in value[-1]:
                    position_in_codon = (i-value[0])%3
                    if position_in_codon ==0:
                        codons.append([i,i+1,i+2])
                    if position_in_codon ==1:
                        codons.append([i-1,i,i+1])
                    if position_in_codon ==2:
                        codons.append([i-2,i-1,i])
            if value[2] == "-":
                for i in value[-1]:
                    position_in_codon = (i-value[1])%3
                    if position_in_codon ==0:
                        codons.append([i,i+1,i+2])
                    if position_in_codon ==1:
                        codons.append([i-1,i,i+1])
                    if position_in_codon ==2:
                        codons.append([i-2,i-1,i])
    return(codons)



In [196]:
print(find_bases(Annotation_dict,"test.fasta"))

[[854, 855, 856], [1166, 1167, 1168], [1654, 1655, 1656], [2050, 2051, 2052], [2873, 2874, 2875], [3269, 3270, 3271], [4088, 4089, 4090], [4223, 4224, 4225], [6102, 6103, 6104], [8792, 8793, 8794], [11330, 11331, 11332], [19827, 19828, 19829], [20250, 20251, 20252], [21218, 21219, 21220], [22948, 22949, 22950], [25459, 25460, 25461], [29366, 29367, 29368], [30882, 30883, 30884], [32155, 32156, 32157], [32164, 32165, 32166], [36182, 36183, 36184], [37355, 37356, 37357], [41536, 41537, 41538], [41758, 41759, 41760], [42301, 42302, 42303], [43343, 43344, 43345], [45566, 45567, 45568], [47956, 47957, 47958], [48865, 48866, 48867], [50418, 50419, 50420], [50877, 50878, 50879], [52501, 52502, 52503], [52501, 52502, 52503], [52531, 52532, 52533], [53011, 53012, 53013], [54102, 54103, 54104], [54672, 54673, 54674], [58822, 58823, 58824], [58828, 58829, 58830], [63229, 63230, 63231], [63982, 63983, 63984], [63982, 63983, 63984], [63982, 63983, 63984], [63985, 63986, 63987], [65203, 65204, 65205

In [197]:
sequences = SeqIO.parse("test.fasta", 'fasta')
extracted_sequences={}
locations = [[854, 855, 856], [1166, 1167, 1168]]
for seq_record in sequences:
    header = seq_record.id
    sequence = str(seq_record.seq)
    
    # Extract sequences based on the provided locations
    extracted_sequences[header] = ''.join(sequence[i - 1] for loc in locations for i in loc)

# Write the extracted sequences to a new FASTA file
with open("output.fasta", 'w') as output_file:
    for header, sequence in extracted_sequences.items():
        output_file.write(f'>{header}\n{sequence}\n')
        